In [1]:
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd 

In [41]:
model = AutoModel.from_pretrained("ibm/MoLFormer-XL-both-10pct", deterministic_eval=True, resume_download=True, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("ibm/MoLFormer-XL-both-10pct", trust_remote_code=True ,resume_download=True)

/home/ubuntu/miniconda3/envs/chem/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [56]:
df = pd.read_csv("/home/ubuntu/data/splits/hi/train.tsv", sep='\t')
val_df = pd.read_csv("/home/ubuntu/data/splits/hi/valid.tsv", sep='\t')

smiles = df.smiles.values.tolist()
val_smiles = val_df.smiles.values.tolist()

In [57]:
inputs = tokenizer(smiles[:], padding=True, return_tensors="pt")
for k,v in inputs.items():
    inputs[k] = v.to("cuda")

val_inputs = tokenizer(val_smiles[:], padding=True, return_tensors="pt")
for k,v in val_inputs.items():
    val_inputs[k] = v.to("cuda")

In [58]:
model = model.to("cuda")

In [59]:
with torch.no_grad():
    train_outputs = model(**inputs)
with torch.no_grad():
    val_outputs = model(**val_inputs)

In [45]:
sum_embeddings = torch.sum(val_outputs.last_hidden_state, 1)

In [47]:
v_df = pd.DataFrame(sum_embeddings.detach().cpu())
v_df['smiles'] = val_smiles
v_df.to_csv('/home/ubuntu/data/molformer_test_embeddings.csv')

In [51]:
e_df = pd.read_csv('/home/ubuntu/data/molformer_embeddings.csv')

In [52]:

from xgboost import XGBRegressor

X, y = e_df[e_df.columns[:-1]].values, df.logS.values
X_val, y_val = v_df[v_df.columns[:-1]].values, val_df.logS.values

# define model
model = XGBRegressor()


AttributeError: 'DataFrame' object has no attribute 'logS'

In [33]:
# fit model
model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [37]:
((model.predict(X) - y)**2).mean()


1.0688312951808567e-06